In [49]:
import os
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
import sys
import os

# Add the src/ folder to Python path
sys.path.append(os.path.abspath("../src"))

from models import *

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.16.2


In [50]:
root_path = "../data/processed"

train_dir = os.path.join(root_path, "train" \
"")
val_dir   = os.path.join(root_path, "val")
test_dir  = os.path.join(root_path, "test")

img_size = (224, 224)
batch_size = 32
seed = 42


In [51]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=True,
    seed=seed,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=False,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=False,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)


Found 4426 files belonging to 5 classes.
Found 948 files belonging to 5 classes.
Found 949 files belonging to 5 classes.
Classes: ['F0', 'F1', 'F2', 'F3', 'F4']


In [52]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)


In [53]:
y_int = []

for _, labels in train_ds.unbatch():
    y_int.append(tf.argmax(labels).numpy())

y_int = np.array(y_int)

weights = compute_class_weight(
    class_weight="balanced",
    classes=np.arange(num_classes),
    y=y_int
)

class_weights = {i: w for i, w in enumerate(weights)}
print("Class weights:", class_weights)


Class weights: {0: 0.5981081081081081, 1: 1.4679933665008291, 2: 1.5949549549549549, 3: 1.4753333333333334, 4: 0.7451178451178451}


2025-11-24 21:13:14.677449: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
"""data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="data_augmentation")"""


In [54]:
"""def build_baseline_cnn(input_shape=(224, 224, 3), num_classes=4):
    inputs = keras.Input(shape=input_shape)

    x = data_augmentation(inputs)
    x = layers.Rescaling(1/255.0)(x)

    for f in [32, 64, 128]:
        x = layers.Conv2D(f, 3, padding="same", activation="relu")(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D()(x)
        x = layers.Dropout(0.25)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    return model"""

baseline_model = build_baseline_cnn(num_classes=num_classes)

baseline_model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

baseline_model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_4 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,453 (501.77 KB)

 Trainable params: 128,005 (500.02 KB)

 Non-trainable params: 448 (1.75 KB)

In [56]:
history_baseline = baseline_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    class_weight=class_weights,
)


Epoch 1/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 45s 317ms/step - accuracy: 0.3443 - loss: 1.5373 - precision_6: 0.4023 - recall_6: 0.1609 - val_accuracy: 0.1350 - val_loss: 1.5988 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00
Epoch 2/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 40s 289ms/step - accuracy: 0.3943 - loss: 1.3735 - precision_6: 0.4684 - recall_6: 0.2207 - val_accuracy: 0.3344 - val_loss: 1.6683 - val_precision_6: 0.3344 - val_recall_6: 0.3344
Epoch 3/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 40s 289ms/step - accuracy: 0.3854 - loss: 1.3664 - precision_6: 0.4560 - recall_6: 0.2237 - val_accuracy: 0.3344 - val_loss: 1.9892 - val_precision_6: 0.3344 - val_recall_6: 0.3344
Epoch 4/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 59s 423ms/step - accuracy: 0.3970 - loss: 1.3348 - precision_6: 0.4679 - recall_6: 0.2338 - val_accuracy: 0.3344 - val_loss: 1.6203 - val_precision_6: 0.3842 - val_recall_6: 0.3344
Epoch 5/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 55s 396ms/step - accuracy: 0.4026 - loss: 1.3370 - precision_6: 0.46

In [33]:
print("Testing on external test dataset…")
baseline_model.evaluate(test_ds)


Testing on external test dataset…
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.4542 - loss: 2.9404 - precision_1: 0.4542 - recall_1: 0.4542


[2.9404494762420654,
 0.45416226983070374,
 0.45416226983070374,
 0.45416226983070374]

In [ ]:
"""def build_resnet50(input_shape=(224, 224, 3), num_classes=4):
    base = tf.keras.applications.ResNet50(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape
    )
    base.trainable = False

    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = tf.keras.applications.resnet50.preprocess_input(x)

    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    return model, base"""

resnet_model, resnet_base = build_resnet50(num_classes=num_classes)

resnet_model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

resnet_model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ data_augmentation   │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ data_augmentatio… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ data_augmentatio… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ data_augmentatio… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 2048)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 512)       │  1,049,088 │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 512)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 5)         │      2,565 │ dropout_9[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [35]:
history_res_frozen = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weights,
)


Epoch 1/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 56s 372ms/step - accuracy: 0.3665 - loss: 2.2387 - precision_2: 0.3812 - recall_2: 0.3294 - val_accuracy: 0.5200 - val_loss: 1.1223 - val_precision_2: 0.5619 - val_recall_2: 0.4641
Epoch 2/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 65s 471ms/step - accuracy: 0.4261 - loss: 2.0019 - precision_2: 0.4405 - recall_2: 0.3918 - val_accuracy: 0.6002 - val_loss: 0.9382 - val_precision_2: 0.6970 - val_recall_2: 0.5338
Epoch 3/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 60s 433ms/step - accuracy: 0.4593 - loss: 1.8864 - precision_2: 0.4728 - recall_2: 0.4202 - val_accuracy: 0.5707 - val_loss: 1.0249 - val_precision_2: 0.6041 - val_recall_2: 0.5264
Epoch 4/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 58s 420ms/step - accuracy: 0.4681 - loss: 1.8589 - precision_2: 0.4822 - recall_2: 0.4349 - val_accuracy: 0.6371 - val_loss: 0.9201 - val_precision_2: 0.7110 - val_recall_2: 0.5918
Epoch 5/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 58s 419ms/step - accuracy: 0.4910 - loss: 1.8539 - precision_2: 0.5074 - rec

In [36]:
resnet_base.trainable = True

for layer in resnet_base.layers[:-30]:
    layer.trainable = False

resnet_model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

history_res_ft = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights,
)


Epoch 1/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 86s 564ms/step - accuracy: 0.5680 - loss: 1.5197 - precision_3: 0.5888 - recall_3: 0.5357 - val_accuracy: 0.7194 - val_loss: 0.7085 - val_precision_3: 0.7600 - val_recall_3: 0.6814
Epoch 2/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 79s 571ms/step - accuracy: 0.6297 - loss: 1.2979 - precision_3: 0.6575 - recall_3: 0.6080 - val_accuracy: 0.7426 - val_loss: 0.6273 - val_precision_3: 0.8089 - val_recall_3: 0.6920
Epoch 3/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 77s 555ms/step - accuracy: 0.6798 - loss: 1.1322 - precision_3: 0.7126 - recall_3: 0.6498 - val_accuracy: 0.7711 - val_loss: 0.5403 - val_precision_3: 0.8453 - val_recall_3: 0.7089
Epoch 4/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 78s 557ms/step - accuracy: 0.7027 - loss: 1.0228 - precision_3: 0.7294 - recall_3: 0.6746 - val_accuracy: 0.7806 - val_loss: 0.5441 - val_precision_3: 0.8384 - val_recall_3: 0.7278
Epoch 5/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 82s 588ms/step - accuracy: 0.7171 - loss: 0.9805 - precision_3: 0.7492 - rec

In [37]:
resnet_model.evaluate(test_ds)


30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 345ms/step - accuracy: 0.9241 - loss: 0.2488 - precision_3: 0.9313 - recall_3: 0.9146


[0.24875938892364502,
 0.9241306781768799,
 0.9313304424285889,
 0.9146469831466675]

In [ ]:
"""def build_effnet(input_shape=(224, 224, 3), num_classes=4):
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape
    )
    base.trainable = False

    inputs = keras.Input(input_shape)
    x = data_augmentation(inputs)
    x = tf.keras.applications.efficientnet.preprocess_input(x)

    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    return model, base"""

eff_model, eff_base = build_effnet(num_classes=num_classes)

eff_model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

eff_model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,378,792 (16.70 MB)

 Trainable params: 329,221 (1.26 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [39]:
history_eff_frozen = eff_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weights,
)


Epoch 1/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 43s 266ms/step - accuracy: 0.3854 - loss: 1.5280 - precision_4: 0.4634 - recall_4: 0.2244 - val_accuracy: 0.4895 - val_loss: 1.0697 - val_precision_4: 0.7778 - val_recall_4: 0.3101
Epoch 2/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 38s 274ms/step - accuracy: 0.4677 - loss: 1.3320 - precision_4: 0.5467 - recall_4: 0.3267 - val_accuracy: 0.5622 - val_loss: 0.9621 - val_precision_4: 0.8466 - val_recall_4: 0.3376
Epoch 3/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 44s 318ms/step - accuracy: 0.4794 - loss: 1.3040 - precision_4: 0.5643 - recall_4: 0.3620 - val_accuracy: 0.5886 - val_loss: 0.9140 - val_precision_4: 0.8614 - val_recall_4: 0.3671
Epoch 4/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 44s 315ms/step - accuracy: 0.5154 - loss: 1.2442 - precision_4: 0.5948 - recall_4: 0.3963 - val_accuracy: 0.6118 - val_loss: 0.8827 - val_precision_4: 0.8732 - val_recall_4: 0.3924
Epoch 5/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 43s 305ms/step - accuracy: 0.5244 - loss: 1.2108 - precision_4: 0.6098 - rec

In [40]:
eff_base.trainable = True

for layer in eff_base.layers[:-40]:
    layer.trainable = False

eff_model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

history_eff_ft = eff_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights,
)


Epoch 1/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 56s 335ms/step - accuracy: 0.5682 - loss: 1.1945 - precision_5: 0.6569 - recall_5: 0.4494 - val_accuracy: 0.6656 - val_loss: 0.7727 - val_precision_5: 0.8016 - val_recall_5: 0.5454
Epoch 2/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 42s 300ms/step - accuracy: 0.5872 - loss: 1.1067 - precision_5: 0.7020 - recall_5: 0.4535 - val_accuracy: 0.6899 - val_loss: 0.7741 - val_precision_5: 0.8331 - val_recall_5: 0.5211
Epoch 3/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 70s 501ms/step - accuracy: 0.5892 - loss: 1.0671 - precision_5: 0.7179 - recall_5: 0.4467 - val_accuracy: 0.6951 - val_loss: 0.7607 - val_precision_5: 0.8401 - val_recall_5: 0.5211
Epoch 4/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 60s 428ms/step - accuracy: 0.6055 - loss: 1.0373 - precision_5: 0.7287 - recall_5: 0.4745 - val_accuracy: 0.6983 - val_loss: 0.7423 - val_precision_5: 0.8433 - val_recall_5: 0.5222
Epoch 5/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 58s 416ms/step - accuracy: 0.6216 - loss: 1.0141 - precision_5: 0.7395 - rec

In [41]:
eff_model.evaluate(test_ds)


30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 191ms/step - accuracy: 0.7724 - loss: 0.5572 - precision_5: 0.8928 - recall_5: 0.6670


[0.5571615099906921, 0.7723919749259949, 0.8928067684173584, 0.667017936706543]

In [46]:
import os
os.makedirs("models", exist_ok=True)

baseline_model.save("models/baseline.keras")
resnet_model.save("models/resnet50.keras")
eff_model.save("models/effnetB0.keras")


In [47]:
import pickle

# Save training histories
with open("models/history_baseline.pkl", "wb") as f:
    pickle.dump(history_baseline.history, f)

with open("models/history_resnet.pkl", "wb") as f:
    pickle.dump(history_res_ft.history, f)

with open("models/history_effnet.pkl", "wb") as f:
    pickle.dump(history_eff_ft.history, f)
